In [1]:
import math
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
def sample(n):
    x = torch.rand(n,1).to(device)
    y = torch.rand(n,1).to(device)
    return x.requires_grad_(True),y.requires_grad_(True)
def lx(u):
    x,y = sample(100)
    cond = (2*x).to(device)
    xy =torch.cat([x,y],dim=1)
    f = u(xy)
    dfdx = torch.autograd.grad(f,x,retain_graph=True,grad_outputs=torch.ones_like(f),create_graph=True,only_inputs=True)[0]
    return loss(dfdx,cond)
def ly(u):
    x,y = sample(100)
    cond = (2*y).to(device)
    xy =torch.cat([x,y],dim=1)
    f = u(xy)
    dfdy = torch.autograd.grad(f,y,retain_graph=True,grad_outputs=torch.ones_like(f),create_graph=True,only_inputs=True)[0]
    return loss(dfdy,cond)

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,1)
        )
    def forward(self,x):
        x = self.net(x)
        return x
device = 'cuda' if torch.cuda.is_available() else 'cpu'

u = model().to(device)
opt = torch.optim.Adam(params=u.parameters())
loss = torch.nn.MSELoss()
Lmin= math.inf
pbar = tqdm(list(range(30000)), desc='PINN_train_process', colour='red')
for i in pbar:
    opt.zero_grad()
    x = 100*torch.rand(1000,1).to(device)
    y = 100*torch.rand(1000,1).to(device)
    f_r = (x**2 + y**2).to(device)
    xy =torch.cat([x,y],dim=1)

    l2 = lx(u) + ly(u)
    f = u (xy).to(device)
    L = loss(f,f_r) + l2
    L.backward()
    opt.step()
    pbar.set_postfix({'Loss':L.cpu().detach().numpy()})
    if Lmin>L:
        Lmin = L
        torch.save(u.state_dict(), './x**2+y**2-models.ckpt')

u = model().to(device)
u.load_state_dict(torch.load('./x**2+y**2-models.ckpt'))
x1 = 100*torch.rand(1000,1).to(device)
y1 = 100*torch.rand(1000,1).to(device)
x1y1 = torch.cat([x1,y1],dim=1)
slove = u(x1y1)
losstest = loss(slove,(x1**2+y1**2))

PINN_train_process: 100%|██████████| 30000/30000 [01:46<00:00, 281.85it/s, Loss=592.03595] 


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1)
slove = slove.detach().numpy()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x1,y1,t)
ax.scatter(x1,y1,slove)
plt.show()